# Hybrid GPU-QPU Optimization

As *PyTorch* natively supports *GPU-accelerated classical processing*, and *Forest* provides quantum hardware access in the form of QPUs, with very little modification, we can run the PyTorch example from last tutorial as a hybrid *GPU-QPU* optimization:

In [ ]:
import pennylane as qml
import torch
from torch.autograd import Variable

qpu = qml.device('forest.qpu', device='Aspen-4-2Q-A')

@qml.qnode(dev, interface='torch')
def circuit(phi, theta):
    qml.RX(theta, wires=0)
    qml.RZ(phi, wires=0)
    return qml.expval.PauliZ(0)

def cost(phi, theta, step):
    target = -(-1)**(step // 100)
    return torch.abs(circuit(phi, theta) - target)**2

phi = Variable(torch.tensor(1., device='cuda'), requires_grad=True)
theta = Variable(torch.tensor(0.05, device='cuda'), requires_grad=True)
opt = torch.optim.Adam([phi, theta], lr = 0.1)

for i in range(400):
    opt.zero_grad()
    loss = cost(phi, theta, i)
    loss.backward()
    opt.step()

<div class="admonition note">

<div class="admonition-title">Note</div>

Access to the hardware *Rigetti* provides needs to be purchased!

</div>

</div>

When using a classical interface that supports GPUs, the QNode will automatically copy any tensor arguments to the CPU before applying them on the specified quantum device. Once done, it will return a tensor containing the QNode result, and automatically copy it back to the GPU for any further classical processing.